<a href="https://colab.research.google.com/github/Tsemogne/efficient-deep-learning/blob/master/onlinemdpabstraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ONLINE ABSTRACTION OF AN ATTACK BY A SWAM OF DRONES**

## Aswam of drones should destroy a target that can detect and destroy them and we want to optimize for a large number of drones as if the number is not large.

We assume that each each drone occupies a position $\mathbf{x} = \left(x_1, x_2\right)$, that a drone is destroyed once it is detected by the target, and that the target is destroyed once any drone is sufficiently close to it.

In [ ]:
# If in the cloud, please uncomment the line below and run this cell
# ! pip install -U https://github.com/PythonOT/POT/archive/master.zip

# 1. **THE GROUND PROBLEM**

The problem is the ground MDP 
$M = \left(
    \mathcal{S},
    \mathcal{A},
    \mathbf{T},
    \mathbf{R}
\right)$
where:

## 1.1. **The States and the Actions**

The state of each vehicule $v$ consists in 5 components: the positional components $s_{v,1}=x_1$ and $x_{v,2}=x_2$, the velocity components $s_{v,3}=\text{\small speed}_1$ and $x_{v,4}=\text{\footnotesize speed}_2$, and an internal component $s_{v,0} \in \left\{0,1\right\}$, $1$ for alive and $1$ for destroyed.
This state is therefore represented by a vector $s_{v} \in \left\{0,1\right\}\times\mathbb{R}^4$.
For the sake of ***discretization***, we assume that the set $\mathbb{R}^2$ of vehicule positions is partitionned in squares 
$$
    \text{Place}_{p_1,p_2} = 
    \left[x_{1,p_1}, x_{1,p_1}+\text{\footnotesize PlaceSkip}\right)
    \times
    \left[x_{2,p_2}, x_{2,p_2}+\text{\footnotesize PlaceSkip}\right),
$$
where $\mathcal{P}=\left\{p_\text{min},\ldots,p_\text{max}\right\}\subseteq\mathbb{N}$,
$p_1$ and $p_2 \in \mathcal{P}$, and $\text{\footnotesize MinSkip} \in \mathbb{R}$, with 
$x_{i,p+1} = x_{i,p} + \text{\footnotesize PlaceSkip}$
for all $i=1,2$.
So the position of a vehicule will always be bottom left corner of the square in which it is.

For the same sake of discretization, the velocity of a vehicule is any vector $\left(\eta_1 \times \text{\footnotesize PlaceSkip},\eta_2 \times \text{\footnotesize PlaceSkip}\right)$,
where $\eta_1,\eta_2 \in \left\{-\Eta, \ldots, 0, \ldots, \Eta\right\}$ satisfy some condition
$\begin{equation}
    \sqrt{\eta_1^2 + \eta_2^2} \leqslant \eta_\text{max}
\end{equation}$
of boundedness, and $\Eta\in\mathbb{N}^*$
and $\eta_\text{max} \leqslant \Eta$
are constant.

Finally, we assume that the state of a vehicule $v$ can be instead represented by a 5-tuple
$$
    s_v = \left(
        s_{v,0}, p_{v,1}, p_{v,2}, \eta_{v,1}, \eta_{v,2}
    \right).
$$

The state of the system is any sequence $s = \left(s_v\right)_{v=0}^{V-1}$ where $s_{v}$ is the state of vehicule $v$ and $V$ is the number of vehicules, or a $V \times 4$ matrix of which each line stands for the state of a vehicule, the first two columns represent the positions of the vehicules and the last two columns represent their velocities.
$$
    s = \left(\begin{array}{ccccc}
        s_{0,0} & p_{0,1} & p_{0,2} & \eta_{0,1} & \eta_{0,2} \\
        \vdots & \vdots & \vdots & \vdots & \vdots\\
        s_{v,0} & p_{v,1} & p_{v,2} & \eta_{v,1} & \eta_{v,2}  \\
        \vdots & \vdots & \vdots & \vdots & \vdots\\
        s_{V-1,0} & p_{V-1,1} & p_{V-1,2} & \eta_{V-1,1} & \eta_{V-1,2} 
    \end{array}\right).
$$

An action on a single vehicule $v$ consists in an attemp to increase or decrease the velocity of $v$. The variation can be represented as a vector of limited norm and the component on each axis should be a multiple of $\text{\small PlaceSkip}$.
So, the set of actions on a single vehicule is
$$\text{\small BasicAction} = \left\{
    \left(\text{\small Pulse}_1,\text{\small Pulse}_2\right) \in \mathbb{Z}^2
    \,\middle|\,
    \sqrt{\text{\small Pulse}_1^2 + \text{\small Pulse}_2^2} \leqslant \text{\small MaximumPulse}
\right\},$$
Where $\text{\small MaximumPulse}$ is a constant integer.
The attempt is successfull provided it does not make the velocity exceed its limit value, i.e.,
$
    \sqrt{\eta_1^2 + \eta_2^2} \leqslant \eta_\text{max}.
$
An action is any $V\times3$ matrix where each line is a basic action, i.e., the action on the corresponding vehicule.

In [ ]:
import torch
import math
global Number_Vehicules, State, PlaceSkip, CoordinateMin, CoordinateMax, PossiblePlace, MaximumShift, MaximumPulse, \
    BasicAction, VehiculesSecurityDistance

def Initialize():
    global Number_Vehicules, State, PlaceSkip, CoordinateMin, CoordinateMax, PossiblePlace, MaximumShift, MaximumPulse, \
        BasicAction, VehiculesSecurityDistance
    print("\n\n\nSIMULATING ATTACK BY DRONES SWAM\n\n")
    3
    Method = int(input("Set parameters to default values ? (0/1)"))
    if Method == 0:
        Number_Vehicules = int(  input("Number of vehicules in the swam:       "))
        print("\nAssume sufficiently closed vehicules will clash against each other")
        VehiculesSecurityDistance = float(input("Security distance of vehicules:    "))
        print("\nPlease border the area as: COORDINATEmin < X < COORDINATEmax, COORDINATEmin < Y < COORDINATEmax")
        CoordinateMin    = float(input("                   COORDINATEmin:  "))
        CoordinateMax    = float(input("                   COORDINATEmax:  "))
        Number_Divisions = int(  input("Number of divisions on each axis:  "))
        MaximumShift     = int(input("Maximum velocity of a vehicule (as a number of divisions): "))
        MaximumPulse     = int(input("Maximum pulse on a vehicule (as a number of divisions):    "))
    else:
        Number_Vehicules =  7
        VehiculesSecurityDistance = 1
        CoordinateMin    = -100
        CoordinateMax    =  100
        Number_Divisions =  1000
        MaximumShift     =  3
        MaximumPulse     =  2
    if CoordinateMin > CoordinateMax:     # It is an error, then should revert the values
        c = CoordinateMax
        CoordinateMax = CoordinateMin
        CoordinateMin = c
    PlaceSkip = (CoordinateMax-CoordinateMin)/(Number_Divisions+1)   # The last range begins one skip befor the maximum coordinate
    PossiblePlace = torch.tensor(range(Number_Divisions), dtype=int)
    # PossibleShift = torch.tensor(range(MaximumShift+1),dtype=int) - int(MaximumShift/2)
    State = InitialState(Number_Vehicules=Number_Vehicules, PossiblePlace=PossiblePlace, MaximumShift=MaximumShift)
    BasicAction = AllActions(MaxPulse=MaximumPulse)
    
def InitialState(Number_Vehicules, PossiblePlace, MaximumShift):
    PMin, PMax = torch.min(PossiblePlace), torch.max(PossiblePlace)
    P1, P2 = PMin.clone(), PMin.clone()
    State = torch.zeros((Number_Vehicules,5),dtype=int)
    for v in range(Number_Vehicules):
        State[v,0], State[v,1], State[v,2], State[v,4] = 1, P1, P2, torch.randint(1,MaximumShift+1,(1,))
        if P1 == PMax:
            P1, P2 = PMin, P2+1
        else:
            P1 += 1
    return State
def AllActions(MaxPulse):
    A = []
    for a in range(-MaxPulse, MaxPulse + 1):
        for b in range(-MaxPulse, MaxPulse + 1):
            if math.sqrt(a**2 + b**2) <= MaxPulse:
                A.append([a, b])
    return torch.tensor(A, dtype=int)

In [ ]:
Initialize()





SIMULATING ATTACK BY DRONES SWAM




In [ ]:
print(BasicAction)

tensor([[-1,  0],
        [ 0, -1],
        [ 0,  0],
        [ 0,  1],
        [ 1,  0]])


In [ ]:
import math
def Motion(State):
    """ Simulate a motion related to a state: each vehicule is shift from its position in the state and according to the velocity expressed in the
    state. NextState is returned. """
    def Clashed(State1, State2, Vehicule1, Vehicule2):
        if State1[Vehicule1,0] == 0 or State1[Vehicule2,0] == 0 \
            or State2[Vehicule1,0] == 0 or State2[Vehicule2,0] == 0:
            return False
        V1P1 = torch.tensor([State1[Vehicule1,1], State1[Vehicule1,2]], dtype=int)
        V1P2 = torch.tensor([State2[Vehicule1,1], State2[Vehicule1,2]], dtype=int)
        V2P1 = torch.tensor([State1[Vehicule2,1], State1[Vehicule2,2]], dtype=int)
        V2P2 = torch.tensor([State2[Vehicule2,1], State2[Vehicule2,2]], dtype=int)
        vect1, vect2 = V2P1-V1P1, V2P2-V1P2
        prog = vect1-vect2
        a = torch.matmul(prog,torch.t(prog)).item()
        b = 2 * (torch.matmul(vect1,torch.t(prog)).item())
        c = torch.matmul(vect1,torch.t(vect1)).item()
        if a == 0:
            MinimumDistance = min(c,b+c)
        else:
            t0 = -b/2/a
            if t0 < 0:
                MinimumDistance = c
            elif t0 > 1:
                MinimumDistance = a+b+c
            else:
                MinimumDistance = a*t0**2 + b*t0 + c
        if MinimumDistance*PlaceSkip < VehiculesSecurityDistance:
            return True
        else:
            return False
        
    NextState = State.clone()
    for v in range(Number_Vehicules):
        if State[v,0] == 1:             # If the vehicule is still alive
            NextState[v,1], NextState[v,2]  =  State[v,1]+State[v,3], State[v,2]+State[v,4]  # Get the next position
            if not(NextState[v,1] in PossiblePlace) or not(NextState[v,2] in PossiblePlace): # If the motion takes the vehicule out of bounds ...
                NextState[v,0] = 0                                                           # then it is detroyed ...
    for v1 in range(Number_Vehicules-1):
        for v2 in range(v1+1,Number_Vehicules):
            if Clashed(State1=State, State2=NextState, Vehicule1=v1, Vehicule2=v2):
                NextState[v1,0] = NextState[v2,0] = 0
    for v in range(Number_Vehicules):
        if NextState[v,0] == 0:
            NextState[v,1] = NextState[v,2] = torch.min(PossiblePlace) - 1
    return NextState

def NewVehiculeVelocity(CurrentVelocityOfVehicule,ActionOnVehicule):
    # Returns the next velocity of a single vehicule after some action on the vehicule
    NextVelocity = CurrentVelocityOfVehicule+ActionOnVehicule
    Norm = math.sqrt((NextVelocity[0]**2 + NextVelocity[1]**2).item())
    if Norm > MaximumShift:
        NextVelocity = (NextVelocity/MaximumShift).int()
    return NextVelocity

def NewVelocity(State, Action):
    # Returns the updated state of velocities of vehicules after some action was performed
    Velocity = State[:,(3,4)]
    for v in range(Number_Vehicules):
        Velocity[v] = NewVehiculeVelocity(CurrentVelocityOfVehicule=Velocity[v], ActionOnVehicule=Action[v])
    return Velocity

def DeterministicTransition(State,Action):
    # Returns the updated state after some action was perfermed, regardless the possible detection by the target
    return Motion(State=NewVelocity(State=State, Action=Action))

import torch

PossiblePlace = torch.tensor([1,3,2,5,-1],dtype=int)
a = torch.tensor([[1,3,3,-1,2],[1,-1,6,2,1]],dtype=int)
b = a[:,(2,3)]
b

tensor([[ 3, -1],
        [ 6,  2]])

In [ ]:
print(NewVehiculeVelocity(CurrentVelocityOfVehicule=State[2,(3,4)],ActionOnVehicule=BasicAction[0]))

tensor([0, 1], dtype=torch.int32)


In [ ]:
print("Current state:\n",State)
print("Basic action:\n",BasicAction)

Current state:
 tensor([[1, 0, 0, 0, 2],
        [1, 1, 0, 0, 3],
        [1, 2, 0, 0, 3],
        [1, 3, 0, 0, 3],
        [1, 4, 0, 0, 1],
        [1, 5, 0, 0, 2],
        [1, 6, 0, 0, 2]])
Basic action:
 tensor([[-2,  0],
        [-1, -1],
        [-1,  0],
        [-1,  1],
        [ 0, -2],
        [ 0, -1],
        [ 0,  0],
        [ 0,  1],
        [ 0,  2],
        [ 1, -1],
        [ 1,  0],
        [ 1,  1],
        [ 2,  0]])
